In [ ]:
# https://hoit1302.tistory.com/159#4.%EC%8B%A4%EC%8A%B5%EC%84%A4%EB%AA%85,%EC%BD%94%EB%93%9C

## Colab 환경 설정하기

In [1]:
!pip install gluonnlp pandas tqdm
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.8.2-py3-none-any.whl (2.5 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.8.1
    Uninstalling transformers-4.8.1:
      Successfully uninstalled transformers-4.8.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kobert 0.2.3 requires transformers<=4.8.1,>=4.8.1, but you have transformers 4.8.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/pub


## github에서 KoBERT 파일 로드 및 KoBERT 모델 불러오기

In [2]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-xblg68g3/kobert-tokenizer_3cc2807d46e54137a8f4210f1a738382
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-xblg68g3/kobert-tokenizer_3cc2807d46e54137a8f4210f1a738382
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT to /tmp/pip-req-build-00y8y1ji
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT' /tmp/pip-req-build-00y8y1ji
  Resolved https://****@github.com/SKTBrain/KoBERT to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Using cached transformers-4.8.1-py3-none-any.whl (2.5 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.8.2
    Uninstalling transformers-4.8.2:
      Successfully uninstalled transformers-4.8.2


In [4]:
import os
from zipfile import ZipFile
import torch
from transformers import BertModel
import gluonnlp as nlp

from kobert import download, get_tokenizer

def get_kobert_model(model_path, vocab_file, ctx="cpu"):
        bertmodel = BertModel.from_pretrained(model_path, return_dict=False)
        device = torch.device(ctx)
        bertmodel.to(device)
        bertmodel.eval()
        vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(
            vocab_file, padding_token="[PAD]"
        )
        return bertmodel, vocab_b_obj

In [5]:
from kobert_tokenizer import KoBERTTokenizer

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)

## 필요한 라이브러리 불러오기

In [6]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm,tqdm_notebook
import pandas as pd
import random

# transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

# GPU사용시
device=torch.device('cuda:0')

## 데이터셋 불러오기

In [7]:
data=pd.read_csv('./Lyrics_final.csv',encoding='utf-8')
data

,Lyrics,label
0,아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 생각하고 늦은 ...,0
1,맛있었다 고 했던 살구빛 샐러드 맛있어서 먹고 싶어 좋아했던 것들을 나에게 자그마한...,0
2,마주치는 눈빛에 아기 아기 왠지 어색해 대체 고개를 숙여봐도 발끝에 맘이 설레 두근...,0
3,어깨동무까진 괜찮아 치마 손잡는 건 위험해 잘 자란 메세 진 괜찮아그래도 자냐는 메...,0
4,햇살이 가득 그댈 향한 가벼운 발걸음 콧노래 부르며 쇼윈도에 비친 모습을 한번 점검...,0
...,...,...
741,반쯤 미쳐가는 망가져 삶넌 내게서 지워가고 부숴버려라 추락한 눈앞에서 멀어지는 속도...,3
742,무궁화 꽃이 피었습니다 빼고 멈춰지담 꽃이 피었습니다 현아 빼고 비켜 도망가 너나 ...,3
743,아예 아예 아예 이렇게 감을 못 잡는 말을 하는지 모르겠어 뭐야 싱긋싱긋 웃으며 치...,3
744,나쁜 계집애 나난 나쁜 계집애 나쁜 계집애 모든 여자가 고난 나난 모든 여자가 곳 ...,3


In [8]:
def new_label(df):
  if df['label']==0 or df['label']==2: #행복(0), 응원(2) -> 0
    return '0'
  elif df['label']==1 or df['label']==3: # 슬픔(1), 분노(3) ->1
    return "1"

data['label2']=data.apply(new_label,axis=1)
data

,Lyrics,label,label2
0,아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 생각하고 늦은 ...,0,0
1,맛있었다 고 했던 살구빛 샐러드 맛있어서 먹고 싶어 좋아했던 것들을 나에게 자그마한...,0,0
2,마주치는 눈빛에 아기 아기 왠지 어색해 대체 고개를 숙여봐도 발끝에 맘이 설레 두근...,0,0
3,어깨동무까진 괜찮아 치마 손잡는 건 위험해 잘 자란 메세 진 괜찮아그래도 자냐는 메...,0,0
4,햇살이 가득 그댈 향한 가벼운 발걸음 콧노래 부르며 쇼윈도에 비친 모습을 한번 점검...,0,0
...,...,...,...
741,반쯤 미쳐가는 망가져 삶넌 내게서 지워가고 부숴버려라 추락한 눈앞에서 멀어지는 속도...,3,1
742,무궁화 꽃이 피었습니다 빼고 멈춰지담 꽃이 피었습니다 현아 빼고 비켜 도망가 너나 ...,3,1
743,아예 아예 아예 이렇게 감을 못 잡는 말을 하는지 모르겠어 뭐야 싱긋싱긋 웃으며 치...,3,1
744,나쁜 계집애 나난 나쁜 계집애 나쁜 계집애 모든 여자가 고난 나난 모든 여자가 곳 ...,3,1


In [9]:
from sklearn.model_selection import train_test_split

Train,Test=train_test_split(data,test_size=0.2,shuffle=True,random_state=111,stratify=data['label'])

In [10]:
dataset_train=[]
for lyric,label in zip(Train['Lyrics'],Train['label']):
  data=[]
  data.append(lyric)
  data.append(str(label))
  dataset_train.append(data)
                       
dataset_test=[]
for lyric,label in zip(Test['Lyrics'],Test['label']):
  data=[]
  data.append(lyric)
  data.append(str(label))
  dataset_test.append(data)

## 입력 데이터셋을 토큰화하기

In [11]:
# Setting parameters
max_len=64
batch_size=32
warmup_ratio=0.1
num_epochs=10
max_grad_norm=1
log_interval=200
learning_rate=5e-5

In [12]:
class BERTDataset(Dataset):
  
  def __init__(self,dataset,sent_idx,label_idx,bert_tokenizer,vocab,max_len,pad,pair):
    transform=nlp.data.BERTSentenceTransform(bert_tokenizer,max_seq_length=max_len,vocab=vocab,pad=pad,pair=pair)
    self.sentences=[transform([i[sent_idx]]) for i in dataset]
    self.labels=[np.int32(i[label_idx]) for i in dataset]

  def __getitem__(self,i):
    return (self.sentences[i]+(self.labels[i],))

  def __len__(self):
    return (len(self.labels))

In [13]:
tok=tokenizer.tokenize
data_train=BERTDataset(dataset_train,0,1,tok,vocab,max_len,True,False)
data_test=BERTDataset(dataset_test,0,1,tok,vocab,max_len,True,False)

In [14]:
data_train[0]

(array([   2, 3093, 6141, 5859, 4778, 7836, 3272,  517, 6473, 7828, 3647,
        6356, 7102, 4239, 5698, 5400, 1597, 6023, 5782, 3977, 6881, 5439,
         517, 6473, 7828, 1958, 6079, 4955, 7398, 7088, 1780, 5808, 5782,
        1193,  993, 6141, 7036, 6855, 2044, 7350, 5400, 1457, 6983, 5047,
        7318, 5842, 4955, 3010, 7788, 4473, 6881, 5439, 3039, 6667, 2056,
        6896, 3220, 7371, 6198, 2563,  517, 7145, 7088,    3], dtype=int32),
 array(64, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [15]:
train_dataloader=torch.utils.data.DataLoader(data_train,batch_size=batch_size,num_workers=5)
test_dataloader=torch.utils.data.DataLoader(data_test,batch_size=batch_size,num_workers=5)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## KoBERT 모델 구현하기

In [16]:
class BERTClassifier(nn.Module):
  def __init__(self,bert,hidden_size=768,num_classes=4,dr_rate=None,params=None):
    super(BERTClassifier,self).__init__()
    self.bert=bert
    self.dr_rate=dr_rate

    self.classifier=nn.Linear(hidden_size,num_classes)
    if dr_rate:
      self.dropout=nn.Dropout(p=dr_rate)

  def gen_attention_mask(self,token_ids,valid_length):
    attention_mask=torch.zeros_like(token_ids)
    for i,v in enumerate(valid_length):
      attention_mask[i][:v]=1
    return attention_mask
  
  def forward(self,token_ids,valid_length,segment_ids):
    attention_mask=self.gen_attention_mask(token_ids,valid_length)
    _,pooler=self.bert(input_ids=token_ids,token_type_ids=segment_ids.long(),
                       attention_mask=attention_mask.float().to(token_ids.device),return_dict=False)
    if self.dr_rate:
      out=self.dropout(pooler)
    return self.classifier(out)

In [17]:
random.seed(111)

# BERT 모델 불러오기
model=BERTClassifier(bertmodel,dr_rate=0.5).to(device)

# optimizer와 schedule 설정
no_decay=['bias','LayerNorm.weight']
optimizer_grouped_parameters=[
    {'params':[p for n,p in model.named_parameters() if not any(nd in n for nd in no_decay)],
     'weight_decay':0.01},
    {'params':[p for n,p in model.named_parameters() if any(nd in n for nd in no_decay)],
     'weight_decay':0.0}
]

optimizer=AdamW(optimizer_grouped_parameters,lr=learning_rate)
loss_fn=nn.CrossEntropyLoss()

t_total=len(train_dataloader)*num_epochs
warmup_step=int(t_total*warmup_ratio)

scheduler=get_cosine_schedule_with_warmup(optimizer,num_warmup_steps=warmup_step,num_training_steps=t_total)

# 정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
  max_vals,max_indices=torch.max(X,1)
  train_acc=(max_indices==Y).sum().data.cpu().numpy()/max_indices.size()[0]
  return train_acc

train_dataloader

## train

In [18]:
random.seed(111)

train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

<ipython-input-18-aa10d7dcc171>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/19 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.3577581644058228 train acc 0.46875
epoch 1 train acc 0.27532894736842106


<ipython-input-18-aa10d7dcc171>:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/5 [00:00<?, ?it/s]

...

epoch 9 test acc 0.6744318181818182


  0%|          | 0/19 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.18881423771381378 train acc 0.96875
epoch 10 train acc 0.9851973684210527


  0%|          | 0/5 [00:00<?, ?it/s]

epoch 10 test acc 0.6619318181818181


In [19]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            print(logits)

#            if np.argmax(logits) == 0:
#                test_eval.append("행복이")
#            elif np.argmax(logits) == 2:
#                test_eval.append("희망이")
            if np.argmax(logits) == 0:
              test_eval.append("행복이")
            elif np.argmax(logits) == 1:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 3:
                test_eval.append("분노가")
            elif np.argmax(logits) == 2:
                test_eval.append("응원이")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [20]:
sentence='힘을 내 이만큼 왔잖아 이것쯤은 별거 아니야 세상을 뒤집자' #응원/긍정
predict(sentence)

[-0.798214  -1.1492938  2.6377957 -0.3625351]
>> 입력하신 내용에서 응원이 느껴집니다.


In [21]:
sentence='너가 떠나가고 혼자 남은 난 어떡하지' #슬픔/부정
predict(sentence)

[-1.3282053   2.580894   -0.22959043 -0.9440793 ]
>> 입력하신 내용에서 슬픔이 느껴집니다.


In [22]:
sentence='너따위가 뭔데' #분노/부정
predict(sentence)

[ 1.7313788 -1.6472417 -1.3827883  1.5554146]
>> 입력하신 내용에서 행복이 느껴집니다.


In [23]:
sentence='기분 좋은 하루의 시작' #행복/긍정
predict(sentence)

[ 2.0621257  -0.9286801   0.25644046 -1.208815  ]
>> 입력하신 내용에서 행복이 느껴집니다.


In [24]:
sentence='요즘 허리가 아파 모임에도 못 나가고 하루 종일 집에만 있어' #슬픔/부정
predict(sentence)

[-0.28481296  0.21027124  1.9159746  -1.6572509 ]
>> 입력하신 내용에서 응원이 느껴집니다.


In [25]:
# strawberry moon - 아이유
sentence='달이 익어가니 서둘러 젊은 피야 민들레 한 송이 들고 사랑이 어지러이 떠다니는 밤이야 날아가 사뿐히 이루렴팽팽한 어둠 사이로 떠오르는 기분 이 거대한 무중력에 혹 휘청해도 두렵진 않을 거야 푸르른 우리 위로'
predict(sentence)

[ 0.42648998 -0.7531459   2.1128356  -1.7203357 ]
>> 입력하신 내용에서 응원이 느껴집니다.
